# Deploy a web app using a Bicep template

In this demo, you'll create a storage account, Azure App Service plan, and a web app using a basic Bicep template. You'll use the Bicep extension for Visual Studio Code.

## Create a Bicep template that contains a storage account

1. Open Visual Studio Code and create a new file called `main.bicep`. 

2. Save the empty file so that Visual Studio Code loads the Bicep tooling.

3. Type the following content into the file, so that you can see how the tooling helps you to write your Bicep files.

```Bicep
resource storageAccount 'Microsoft.Storage/storageAccounts@2022-05-01' = {
  name: 'otp20221227sa'
  location: 'westeurope'
  sku: {
    name: 'Standard_LRS'
  }
  kind: 'StorageV2'
  properties: {
    accessTier: 'Hot'
  }
}
```

4. The Bicep extension for Visual Studio Code understands the resources you're defining in your template, and it lists the available properties and values that you can use. Use `Ctrl+Space` to trigger the suggestions. 

5. The name of the storage account needs to be **globally unique**. Make sure the name is all lowercase, without any special characters, and fewer than 24 characters.

6. Save the changes to the file.



## Deploy the Bicep template to Azure

To deploy this template to Azure, you need to sign in to your Azure account. You can use Azure PowerShell or Azure CLI.

In [ ]:
# Connect-AzAccount 
# If you have a managed identity enabled
# Connect-AzAccount -Identity

# $context = Get-AzSubscription -SubscriptionName <Your Subscription>
# Set-AzContext $context

# Set the default resource group
Set-AzDefault -ResourceGroupName 'otp-rg'

# Deploy the template to Azure
New-AzResourceGroupDeployment -TemplateFile ./main.bicep

# Verify the deployment
Get-AzResourceGroupDeployment -ResourceGroupName 'otp-rg' | Format-Table

In [ ]:
# az login
# If you have a managed identity enabled
# az login --identity

# az account set --subscription <Your Subscription>

# Set the default resource group
az configure --defaults group='otp-rg'

# Deploy the template to Azure
az deployment group create --template-file ./main.bicep

# Verify the deployment
az deployment group list --output table

## Add an App Service plan and a web app to your Bicep template

Now you're ready to deploy more resources, including a dependency.
In this task, you add an App Service plan and a web app to the Bicep template.

1. In the `main.bicep` file in Visual Studio Code, add the following code to the bottom of the file:

```
resource appServicePlan 'Microsoft.Web/serverFarms@2022-03-01' = {
  name: 'otp2022-plan'
  location: 'westeurope'
  sku: {
    name: 'F1'
    tier: 'Free'
  }
}

resource webApp 'Microsoft.Web/sites@2022-03-01' = {
  name: 'otp20221227-app'
  location: 'westeurope'
  properties: {
    serverFarmId: appServicePlan.id
    httpsOnly: true
  }
}
```

2. Update the name of the App Service app to be unique.
Make sure the name is all lowercase, contains only alphanumerics and hyphens, doesn't start or end with a hyphen, and has 2 to 60 characters.

3. Save the changes to the file.

In [ ]:
# Add an App Service plan and a web app to the main.bicep file
@"

resource appServicePlan 'Microsoft.Web/serverFarms@2022-03-01' = {
  name: 'otp2022-plan'
  location: 'westeurope'
  sku: {
    name: 'F1'
    tier: 'Free'
  }
}

resource webApp 'Microsoft.Web/sites@2022-03-01' = {
  name: 'otp20221227-app'
  location: 'westeurope'
  properties: {
    serverFarmId: appServicePlan.id
    httpsOnly: true
  }
}
"@ | Out-File -FilePath ./main.bicep -Append

In [ ]:
# Deploy the updated Bicep template and check your deployment
New-AzResourceGroupDeployment -TemplateFile ./main.bicep
Get-AzResourceGroupDeployment -ResourceGroupName 'otp-rg' | Format-Table

In [ ]:
# Deploy the updated Bicep template and check your deployment
az deployment group create --template-file ./main.bicep
az deployment group list --output table

## Add the location and resource name parameters

In the `main.bicep` file in Visual Studio Code, add the following code to the top of the file:

```
param location string = resourceGroup().location
param storageAccountName string = 'otp2022${uniqueString(resourceGroup().id)}'
param appServiceAppName string = 'otp2022${uniqueString(resourceGroup().id)}'

var appServicePlanName = 'otp2022-plan'
```

You're using expressions that include string interpolation, the uniqueString() function, and the resourceGroup() function to define default parameter values. Someone deploying this template can override the default parameter values by specifying the values at deployment time, but they can't override the variable values.

Storage accounts and App Service apps need globally unique names, but App Service plans need to be unique only within their resource group.

In [ ]:
$prependLines = @(@'
param location string = resourceGroup().location
param storageAccountName string = 'otp2022${uniqueString(resourceGroup().id)}'
param appServiceAppName string = 'otp2022${uniqueString(resourceGroup().id)}'

var appServicePlanName = 'otp2022-plan'

'@)

$prependLines + (Get-Content ./main.bicep) | Set-Content .\main.bicep

Find the places within the resource definitions where the location and name properties are set, and update them to use the parameter values. The resource definitions within your Bicep file should look like this:

```
resource storageAccount 'Microsoft.Storage/storageAccounts@2022-05-01' = {
  name: storageAccountName
  location: location
  sku: {
    name: 'Standard_LRS'
  }
  kind: 'StorageV2'
  properties: {
    accessTier: 'Hot'
  }
}

resource appServicePlan 'Microsoft.Web/serverFarms@2022-03-01' = {
  name: appServicePlanName
  location: location
  sku: {
    name: 'F1'
    tier: 'Free'
  }
}

resource webApp 'Microsoft.Web/sites@2022-03-01' = {
  name: appServiceAppName
  location: location
  properties: {
    serverFarmId: appServicePlan.id
    httpsOnly: true
  }
}
```

Save the changes to the file.

## Automatically set the SKUs for each environment type

Add the following Bicep parameter below the parameters that you created in the previous task:

```
@allowed([
  'dev'
  'prod'
])
param environment string
```

You're defining a parameter with a set of allowed values, but you're not specifying a default value for this parameter.

Below the line that declares the `appServicePlanName` variable, add the following variable definitions:

```
var storageAccountSkuName = (environment == 'prod') ? 'Standard_GRS' : 'Standard_LRS'
var appServicePlanSkuName = (environment == 'prod') ? 'P2_v3' : 'F1'
var appServicePlanTierName = (environment == 'prod') ? 'PremiumV3' : 'Free'
```

You're setting these variables' values by using the ternary operator to express some if/then/else logic.

Find the places within the resource definitions where the SKU properties are set, and update them to use the variable values. After you're finished, the resource definitions in your Bicep file should look like this:

```
resource storageAccount 'Microsoft.Storage/storageAccounts@2022-05-01' = {
  name: storageAccountName
  location: location
  sku: {
    name: storageAccountSkuName
  }
  kind: 'StorageV2'
  properties: {
    accessTier: 'Hot'
  }
}

resource appServicePlan 'Microsoft.Web/serverFarms@2022-03-01' = {
  name: appServicePlanName
  location: location
  sku: {
    name: appServicePlanSkuName
    tier: appServicePlanTierName
  }
}

resource webApp 'Microsoft.Web/sites@2022-03-01' = {
  name: appServiceAppName
  location: location
  properties: {
    serverFarmId: appServicePlan.id
    httpsOnly: true
  }
}
```

Save the changes to the file.

In [ ]:
# Deploy the updated Bicep template
New-AzResourceGroupDeployment -TemplateFile main.bicep -environment dev
Get-AzResourceGroupDeployment -ResourceGroupName 'otp-rg' | Format-Table

In [ ]:
# Deploy the updated Bicep template
az deployment group create --template-file ./main.bicep --parameters environment=dev
az deployment group list -o table

## Add a new module file

Create a new folder called `modules` in the same folder where you created your `main.bicep` file. In the `modules` folder, create a file called `appService.bicep`. Save the file.

In [ ]:
mkdir modules
cd modules
New-Item appService.bicep

Add the following content into the appService.bicep file:

```
param location string
param appServiceAppName string

@allowed([
  'dev'
  'prod'
])
param environment string

var appServicePlanName = 'otp2022-plan'
var appServicePlanSkuName = (environment == 'prod') ? 'P2_v3' : 'F1'
var appServicePlanTierName = (environment == 'prod') ? 'PremiumV3' : 'Free'

resource appServicePlan 'Microsoft.Web/serverFarms@2022-03-01' = {
  name: appServicePlanName
  location: location
  sku: {
    name: appServicePlanSkuName
    tier: appServicePlanTierName
  }
}

resource webApp 'Microsoft.Web/sites@2022-03-01' = {
  name: appServiceAppName
  location: location
  properties: {
    serverFarmId: appServicePlan.id
    httpsOnly: true
  }
}
```

You've copied the parameters and variables from your main.bicep template, because the appService.bicep template needs to be self-contained.

Save the changes to the file.

In [ ]:
@"
param location string
param appServiceAppName string

@allowed([
  'dev'
  'prod'
])
param environment string

var appServicePlanName = 'otp2022-plan'
var appServicePlanSkuName = (environment == 'prod') ? 'P2_v3' : 'F1'
var appServicePlanTierName = (environment == 'prod') ? 'PremiumV3' : 'Free'

resource appServicePlan 'Microsoft.Web/serverFarms@2022-03-01' = {
  name: appServicePlanName
  location: location
  sku: {
    name: appServicePlanSkuName
    tier: appServicePlanTierName
  }
}

resource webApp 'Microsoft.Web/sites@2022-03-01' = {
  name: appServiceAppName
  location: location
  properties: {
    serverFarmId: appServicePlan.id
    httpsOnly: true
  }
}
"@ | Out-File -FilePath ./appService.bicep

code c:/zero2hero/modules/appService.bicep

## Add a reference to the module from the parent template

Now that you have a complete module to deploy the App Service resources, you can refer to the module within the parent template.

In the `main.bicep` file, delete the App Service resources and the `appServicePlanName`, `appServicePlanSkuName`, and `appServicePlanTierName` variable definitions. Don't delete the App Service-related parameters, because you still need them. Also, don't delete the storage account parameters, variable, or resources.

At the bottom of the `main.bicep` file, add the following Bicep code:

```
module appService 'modules/appService.bicep' = {
  name: 'appService'
  params: {
    location: location
    appServiceAppName: appServiceAppName
    environment: environment
  }
}
```

You're specifying the parameters for your module by referencing the parameters in the parent template.

Save the changes to the file.

## Add the host name as an output

Add the following Bicep code at the bottom of the `appService.bicep` file:

```
output webAppHostName string = webApp.properties.defaultHostName
```

This code is declaring that an output for this module, which will be named `appServiceAppHostName`, will be of type string. The output will take its value from the `defaultHostName` property of the App Service app.

Save the changes to the file.

This output is declared within a Bicep file that we'll use as a module, so it's going to be available only to the parent template. You also need to return the output to the person who's deploying the template.

Open the `main.bicep` file and add the following code at the bottom of the file:

```
output webAppHostName string = appService.outputs.webAppHostName
```

This output is declared in a similar way to the output in the module. But this time, you're referencing the module's output instead of a resource property.

Save the changes to the file.

In [ ]:
# Deploy the updated Bicep template
cd c:/zero2hero
New-AzResourceGroupDeployment -TemplateFile ./main.bicep -environment dev

In [ ]:
# Deploy the updated Bicep template
cd c:/zero2hero
az deployment group create --template-file ./main.bicep --parameters environment=dev

In [ ]:
# Open the default App Service welcome page
start  ('https://{0}' -f (Get-AzResourceGroupDeployment -ResourceGroupName otp-rg | where DeploymentName -eq main | Select-Object -ExpandProperty outputs).values.value)